Import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import os

import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from torch import optim
from PIL import Image
from torchvision import transforms, models

Define the directories and read the csv files

In [ ]:
data_dir = "../input/dog-breed-identification/"
train_fol = os.path.join(data_dir, "train")
test_fol = os.path.join(data_dir, "test")
train_labels = pd.read_csv(os.path.join(data_dir,"labels.csv"))
sample_sub = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))

BREEDS = list(train_labels['breed'].unique())
BREEDS.sort()
BREEDS_TO_CLASS = {x:y for x,y in zip(BREEDS,range(len(BREEDS)))}
CLASS_TO_BREED = {y:x for y,x in enumerate(BREEDS_TO_CLASS)}

Image transformations

In [ ]:
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.476, 0.452, 0.392],
                             std=[0.235, 0.231, 0.229])
    ])

class DogDataset(Dataset):
    def __init__(self, image_folder, labels_matrix, transformers = data_transform, training = True):
        self.image_folder = image_folder
        self.labels_matrix = labels_matrix
        self.tf = transformers
        self.training = training
        
    def __len__(self):
        return self.labels_matrix.shape[0]

    def __getitem__(self, idx):
        image_name_i = self.labels_matrix.iloc[idx]['id'] + '.jpg'
        image_i  = Image.open(os.path.join(self.image_folder, image_name_i))
        image_i = self.tf(image_i)
        if self.training == True:
            label_i = self.labels_matrix.iloc[idx,1:].values.argmax()
            return image_i, label_i
        else:
            return image_i

Get the pretrained Resnet50 model and freeze the trained parameters. Change the final layer to match the number of classes. 

In [ ]:
def DogBreedPredictor(pretrained = True):
    model = models.resnet50(pretrained=pretrained)
    for param in model.parameters():
        param.requires_grad = False
    in_fea = model.fc.in_features
    model.fc = torch.nn.Linear(in_fea, 120)    
    return model

Method to train the Resnet50 model

In [ ]:
def train_model(model, data_loaders, optimizer, loss_criteria, scheduler, epochs, use_gpu=True):
    begin = dt.datetime.now() #Start the timer
    training_loss = [] #to plot loss curve
    val_loss = [] #to plot loss curve
    if use_gpu and torch.cuda.is_available():
        print("Using GPU")
        model = model.cuda()
    best_model_wts = model.state_dict()
    best_val_acc = 0.0
    for epoch in range(epochs):
        #train
        train_cum_loss_epoch = 0.0
        train_correct_predictions = 0.0
        model.train()
        for inputs, labels in data_loaders['train']:
            if use_gpu and torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_criteria(outputs, labels)
            loss.backward()
            optimizer.step()
            
            _, predictions = torch.max(outputs,1)
            train_correct_predictions+=torch.sum(predictions==labels).item()
            train_cum_loss_epoch+=loss.item()
        scheduler.step()
        
        #validation
        val_cum_loss_epoch = 0.0
        val_correct_predictions = 0.0
        model.eval()
        for inputs, labels in data_loaders['val']:
            if use_gpu and torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()

            outputs = model(inputs)
            _, predictions = torch.max(outputs,1)
            val_correct_predictions+=torch.sum(predictions==labels).item()
            val_cum_loss_epoch+=loss_criteria(outputs, labels).item()
            
        #metrics
        train_mean_epoch_loss = train_cum_loss_epoch / data_loaders['train'].batch_size
        val_mean_epoch_loss = val_cum_loss_epoch / data_loaders['val'].batch_size
        train_accuracy = train_correct_predictions / len(data_loaders['train'].dataset)
        val_accuracy = val_correct_predictions / len(data_loaders['val'].dataset)
        training_loss.append(train_mean_epoch_loss)
        val_loss.append(val_mean_epoch_loss)
        #print
        print('Epoch [{}/{}] train loss: {:.4f} train acc: {:.4f} ' 
              'val loss: {:.4f} val acc: {:.4f}'.format(
                epoch + 1 , epochs,
                train_mean_epoch_loss, train_accuracy, 
                val_mean_epoch_loss, val_accuracy))
        
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            opt_epoch = epoch + 1
            best_model_wts = model.state_dict()
    print("Running time: ", dt.datetime.now()-begin)
    print('Best val Acc: {:4f} and optimal epoch: {}'.format(best_val_acc, opt_epoch))
    #load best weights
    model.load_state_dict(best_model_wts)
    return model, training_loss, val_loss

In [ ]:
def submit_model(model, data_loader, use_gpu=True):
    begin = dt.datetime.now() #Start the timer
    sub_df = pd.DataFrame(index = sample_sub.id.tolist(), columns=sample_sub.keys().tolist()[1:])
    sub_val = []
    if use_gpu and torch.cuda.is_available():
        print("Using GPU")
        model = model.cuda()
    model.eval()    
    with torch.no_grad():
        for inputs in data_loader:
            if use_gpu and torch.cuda.is_available():
                inputs = inputs.cuda()
            outputs = model(inputs)
            outputs = torch.nn.functional.softmax(outputs,dim=1)
            sub_val.append(outputs.data.cpu().numpy())
    sub_val = np.concatenate(sub_val)
    sub_df.loc[:,:] = sub_val
    return sub_df    

In [ ]:
#create a dataframe for trainset similar to submission dataframe
train_labels['class'] = 1
labels_matrix = train_labels.pivot('id','breed','class').reset_index().fillna(0)

#split the training data into train and validation
train_labels, val_labels = np.split(labels_matrix, [int(len(labels_matrix)*0.8)], axis=0)

#Pytorch dataset and dataloader
train_ds = DogDataset(train_fol, train_labels)
val_ds = DogDataset(train_fol, val_labels)

train_loader = DataLoader(train_ds, batch_size=30, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=30, shuffle=False)

dataloaders = {'train':train_loader,'val':val_loader}

In [ ]:
model = DogBreedPredictor(True)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001) #initialize the optimizer
#optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
loss_criteria = torch.nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=7, gamma=0.1, last_epoch=-1)

In [ ]:
modeltr, traing_curve, val_curve = train_model(model, dataloaders, optimizer, loss_criteria, scheduler, 25, True)

In [ ]:
sub_ds = DogDataset(test_fol, sample_sub, data_transform, False)
sub_dl = DataLoader(sub_ds, batch_size=30, shuffle=False)
sub_df = submit_model(model, sub_dl)

In [ ]:
sub_df_names = sub_df.idxmax(axis=1)
sub_df_names = pd.DataFrame({'id': sub_df_names.index, 'breed': sub_df_names.values})

In [ ]:
sub_df.to_csv("submission.csv",index=True)

In [ ]:
sub_df_names.to_csv("submission_names.csv",index=True)

In [ ]:
torch.save(model.state_dict(), "model3.pt")

In [ ]:
model = DogBreedPredictor()

In [ ]:
model.load_state_dict(torch.load("model3.pt"))